#### This notebook takes predicted images from Uni-EM and does post-prediction analyses.
It imports pre-segmented 2D EM data from Uni-EM and outputs the following measures as a pandas dataframe and excel sheet:


##### These measures are output for every fiber AND myelin:

'label': unique ID for every cell found.

'area': area (in pixels) of the given cell

'centroid': x and y coordinats of the centroid of each cell

'axis_major_length': equivalent oval major length

'axis_minor_length': equivalent oval minor length

'eccentricity': Value between 0 and 1 that defines the non-circularity of a structure. The close it is to 0, the closer it is to a circle. An eccentricity of 1 means it is a parabola. Eccentricity here is defined as the ratio of the focal distance (distance between focal points) over the major axis length of the ellipse with the same secondary moment.

'orientation': Orientation of the structure in rad, value between -1/2pi and +1/2pi.

'equivalent_diameter_area: diameter of the circle with an equivalent area to the cell

'slice': The slice number to extract this cell from the image


##### These measures are output for every single cell:

'file': The filename of the image this cell came from

'gratio': approximate g ratio of the given cell, here defined as axis_minor_length(fiber)/axis_minor_length/(myelin)

'orientation_mean_deg': Mean orientation between the fiber and myelin (in degrees)

'eccentricity_mean': Mean eccentricity between the fiber and myelin

for more info on the extracted values, see https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops


In [ ]:
# import dependencies
import numpy as np
import tifffile as tf
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
import pandas as pd
import math as m
import cv2
import skimage
import tqdm
from skimage.measure import regionprops, regionprops_table
from skimage.morphology import disk
from skimage.feature import peak_local_max
from os import listdir
from os.path import isfile, join
import colorcet as cc
import plotly.express as px
import plotly.io as pio
import re
%matplotlib inline


In [ ]:
def resolve_undersegmentation(outer_labels,inner_labels):
    '''
    resolves undersegmented cells with 'kissing' cells. This function assumes that inner_labels never touch, but outer_labels do.
    Each corresponding outer area must touch each corresponding inner area.

    outer_labels = outer labels with undersegmented kissing cells that need to be seperated
    inner_labels = inner labels without undersegmented kissing cells

    returns: Two arrays of the same shape, with uniform labels across both images. 
    '''
    from skimage.segmentation import watershed
    from skimage.feature import peak_local_max
    outer_labels = outer_labels.astype('bool')
    inner_labels = inner_labels.astype('bool')
    # do distance transformation of combined binary image of outer+inner
    seg_dist = ndi.distance_transform_edt(outer_labels+inner_labels)
    # Generate the markers as local maxima of the distance to the background
    coords = peak_local_max(seg_dist, footprint=np.ones((3, 3)), labels=ndi.label(inner_labels)[0], num_peaks_per_label=1)
    # initializie empty mask 
    mask = np.zeros(seg_dist.shape, dtype=bool)
    # insert maximum points into the empty array
    mask[tuple(coords.T)] = True
    # label each maximum
    markers, n = ndi.label(mask)
    # print("number of cells: " + str(n))
    # perform watershed on outer_labels and inner_labels combined
    outer_cells = watershed(-seg_dist, markers, mask=(outer_labels+inner_labels))
    inner_cells = np.copy(outer_cells)
    # sort the cells (which are now labeled with the same label inner and outer) back into inner and outer labels
    inner_cells[inner_labels==0] = 0
    outer_cells[inner_labels==True] = 0
    return outer_cells,inner_cells

def keep_largest_structure(boolean_array):
    from scipy import ndimage
    # Label each connected component in the boolean array
    labeled_array, num_features = ndimage.label(boolean_array)

    # Calculate the size of each labeled component
    component_sizes = np.bincount(labeled_array.ravel())

    # Find the index of the largest component
    largest_component_index = np.argmax(component_sizes[1:]) + 1

    # Create a boolean mask to keep only the largest component
    largest_component_mask = labeled_array == largest_component_index

    # Apply the mask to the boolean array
    boolean_array[largest_component_mask] = True
    boolean_array[~largest_component_mask] = False

    return boolean_array

def find_numbers(string):
    pattern = r'\d{1,4}'  # Regular expression pattern for matching numbers with 1 to 4 digits
    numbers = re.findall(pattern, string)
    numbers = [int(num) for num in numbers]  # Convert numbers to integers
    return numbers

# define qualitative colormap
glasbey = cc.cm.glasbey_dark_r
glasbey.set_under(color="black")

In [ ]:
# user inputs

# path to segmented images
# path_raw_predictions = (r"G:\AG_Morawski\Philip\EM\Uni-EM\test_img_output\\")
path_raw_predictions = (r"G:\AG_Morawski\Philip\EM\SWM_Directionality\3_predicted\\")
prediction_files = [f for f in listdir(path_raw_predictions) if isfile(join(path_raw_predictions, f))]

troubleshoot_small_big_gratios = False # Troubleshooting purposes. Only set to True if you get weird (lots of very small (<0.1) and big (>0.9)) gratios
plot_all = False # if False, only plots plots that are saved. True is recommended only for troubleshooting purposes

# path to raw images (optional)
# open_path_raw = (
#                 r"G:/AG_Morawski/Philip/EM/Uni-EM/test_img/test_img_0.png", #CC data
#                 r"G:/AG_Morawski/Philip/EM/HH4_20230419_processed/Bild_040_raw.png", #SWM Data
#                 )

# open_path_raw_images = [f for f in listdir(open_path_raw) if isfile(join(open_path_raw, f))]
    
# path to where you want your results to be
# path_results = (r"G:\AG_Morawski\Philip\EM\Uni-EM\test_img_post_post_uni-em\\")
path_results = (r"G:\AG_Morawski\Philip\EM\SWM_Directionality\4_results\\")

In [ ]:
i = 0
for file in tqdm.tqdm(prediction_files):
    # img = cv2.imread(path_raw_predictions + file)
    #img_raw = cv2.imread(open_path_raw[0])
    img = skimage.io.imread(path_raw_predictions + file)

    # lose two dimensions, since its a grayscale img 
    # img = img[:,:,0]
    
    if plot_all == True:
        # plot raw image
        fig, ax = plt.subplots(ncols=1,figsize=(8,8))
        ax.imshow(img, cmap='viridis')
        ax.set_title('Uni-EM Presegmentation')
        plt.show()
        plt.hist(img.ravel(), bins=256, range=(0, 255),)
        plt.show()

    # split different labels
    outer = np.zeros_like(img)
    inner = np.zeros_like(img)
    outer[img>55] = 1
    inner[img<40] = 1
    inner[img<24] = 0
    # outer[img>=50] = 1
    # inner[img<50] = 1
    # inner[img<24] = 0
    if plot_all == True:
        fig, axs = plt.subplots(ncols=2,figsize=(12,12))
        axs[0].imshow(outer, cmap='gray')
        axs[1].imshow(inner, cmap='gray')
        axs[0].set_title('Outer channel of pre-segmentation')
        axs[1].set_title('Inner channel of pre-segmentation')
        plt.show()

    # binary opening to get rid of small speckles
    inner = ndi.binary_opening(inner,structure=disk(2))

    # fill holes
    inner = ndi.binary_fill_holes(inner)

    # binary opening to get rid of small speckles
    inner = ndi.binary_opening(inner,structure=disk(6))

    # dilate inner, then restrict it to everywhere where outer isnt true. 
    # This is to make sure they are in contact and can be seperated by watershed later on
    inner = ndi.binary_dilation(inner,structure=disk(3))
    inner[outer==True]=False

    if plot_all == True:
        # plot overlay of inner + outer as sanity check
        plt.title('after dilation of inner & restriction to outer channel')
        plt.imshow(np.ma.array(inner,mask=inner==0),interpolation='None',cmap='tab20')
        plt.imshow(np.ma.array(outer,mask=outer==0),interpolation='None',cmap='gray')
        plt.show()

    # crop for quick troubleshooting purposes
    # inner = inner[1000:3000,0:2000]
    # outer = outer[1000:3000,0:2000]

    # label structures. This function gives every seperated structure a unique integer as identifier. 
    mask_labeled, n_struct = ndi.label(inner)

    #### Cleanup of segmented data
    # remove cells that intersect with the border of the image
    border_mask = np.zeros(mask_labeled.shape, dtype=bool)
    border_mask = ndi.binary_dilation(border_mask,iterations=5, border_value=5) #should not need iterations=5, but sample data is weird

    for id in np.unique(mask_labeled):
        #keep only current cell as temporary mask
        current_id_mask = mask_labeled==id 

        # check if it overlaps with the border mask
        overlap = np.logical_and(border_mask,current_id_mask)
        overlap_n = np.sum(overlap)

        if overlap_n > 0:
            mask_labeled[mask_labeled==id] = 0


    # re-label structures so all cells have continous numbering
    mask_labeled[mask_labeled>0] = 1
    mask_labeled, n_struct = ndi.label(mask_labeled)

    if plot_all == True:
        plt.axis("off")
        plt.imshow(mask_labeled.astype('bool'),cmap='gray')
        plt.show()
    
    #### Find corresponding outer cells for every inner cell
    # Here, we find corresponding outer structure to each inner structure. 
    # fuse arrays, then flood fill from the inside and then seperate back into single channels.
    # This gives every cell a homogeneous number in both channels.
    outer_labeled,inner_labeled = resolve_undersegmentation(outer,mask_labeled.astype('bool'))

    # at this point, some of the cells do not have myelin. 
    # Since we only look for myelinated cells, these are filtered out here.
    idx_list = []
    for idx in np.unique(inner_labeled):
        if (outer_labeled==idx).max() == False:
            inner_labeled[inner_labeled==idx] = 0
            idx_list.append(idx)
    print('deleted cells because they don\'t have myelin: ' + str(idx_list))

    #re-label so the labels are uniform again.
    outer_labeled,inner_labeled = resolve_undersegmentation(outer_labeled.astype('bool'),inner_labeled.astype('bool'))

    #### Post-Process filtering
    # if there are multiple independent labels with the same labelling number,
    # keep only the biggest one - Inner
    for id in np.unique(inner_labeled):
        if id == 0:
            continue
        current_id_mask = inner_labeled==id 
        mask_labeled, n = ndi.label(current_id_mask)
        if n > 1:
            #delete all but the biggest one from this image and from pred_inner_labeled
            current_id_kept = keep_largest_structure(current_id_mask)
            #correct the value of the boolean array to the current id
            current_id_kept = (current_id_kept*id).astype('uint16')
            #re-insert the remaining one to pred_inner_labeled
            inner_labeled[inner_labeled==id] = 0
            inner_labeled = inner_labeled + current_id_kept        
            if plot_all == True:
                print ("Removed " + str(n-1) + " inner duplicates from id #" + str(id))

    # if there are multiple independent labels with the same labelling number, keep only the biggest one - Outer
    for id in np.unique(outer_labeled):
        if id == 0:
            continue
        current_id_mask = outer_labeled==id 
        mask_labeled, n = ndi.label(current_id_mask)
        if n > 1:
            #delete all but the biggest one from this image and from pred_inner_labeled
            current_id_kept = keep_largest_structure(current_id_mask)
            #correct the value of the boolean array to the current id
            current_id_kept = (current_id_kept*id).astype('uint16')
            #re-insert the remaining one to pred_inner_labeled
            outer_labeled[outer_labeled==id] = 0
            outer_labeled = outer_labeled + current_id_kept        
            if plot_all == True:
                print ("Removed " + str(n-1) + " outer duplicates from id #" + str(id))

    # if fibers are unreasonably small, remove them from further analysis.
    for id in np.unique(inner_labeled):
        n_px = (np.count_nonzero(inner_labeled[inner_labeled==id]))
        if n_px < 5:
            inner_labeled[inner_labeled==id]=0
            outer_labeled[outer_labeled==id]=0
            print('deleted cell#' + str(id))

    #plot. there should now not be a single cell without myelin.
    # plot overlay of inner + outer as sanity check
    # generate 2px outline of inner and outer area
    outer_eroded = ndi.binary_erosion(outer_labeled,structure=disk(3))
    ero = np.logical_xor(outer_labeled.astype('bool'),outer_eroded)
    vmin = 0
    vmax = inner_labeled.max()
    plt.figure(figsize=(12,12))
    plt.title('Labelling of each cell instance')
    plt.imshow(np.ma.array(inner_labeled + outer_labeled, mask=(inner_labeled + outer_labeled) == 0), cmap=glasbey,vmin=vmin,vmax=vmax)
    plt.imshow(np.ma.array(ero, mask=(ero) == 0), interpolation='None', cmap='gray',)
    # plt.axis('off')
    plt.savefig(path_results + file + "_labeled_cells.png",dpi=500)
    plt.show()

    table = regionprops_table(label_image=inner_labeled,
                              # intensity_image=img_raw,
                              properties=('label',
                                          'area',
                                          'centroid',
                                          'axis_major_length',
                                          'axis_minor_length',
                                          'eccentricity',
                                          'orientation',
                                          'slice',
                                          # 'extent',
                                          'equivalent_diameter_area',))
                                        #   'intensity_max',
                                        #   'intensity_min',
                                        #   'intensity_mean',))
                                        #   'image',))
    measurements_inner = pd.DataFrame(table)

    if plot_all == True:
        # plot centroids, measurements, major and minor axis on image to sanity check
        regions = regionprops(inner_labeled)

        fig, ax = plt.subplots(figsize=(8,8))
        ax.imshow(inner_labeled.astype('bool'), cmap='gray')

        for props in regions:
            y0, x0 = props.centroid
            orientation = props.orientation
            x1 = x0 + m.cos(orientation) * 0.5 * props.axis_minor_length
            y1 = y0 - m.sin(orientation) * 0.5 * props.axis_minor_length
            x2 = x0 - m.sin(orientation) * 0.5 * props.axis_major_length
            y2 = y0 - m.cos(orientation) * 0.5 * props.axis_major_length

            ax.plot((x0, x1), (y0, y1), '-r', linewidth=2.5)
            ax.plot((x0, x2), (y0, y2), '-r', linewidth=2.5)
            ax.plot(x0, y0, '.g', markersize=10)

            minr, minc, maxr, maxc = props.bbox
            bx = (minc, maxc, maxc, minc, minc)
            by = (minr, minr, maxr, maxr, minr)
            ax.plot(bx, by, '-b', linewidth=1.5)

        ax.set_axis_off()
        ax.axis((0, 1000, 1000, 0))
        plt.show()

    table = regionprops_table(label_image=outer_labeled,
                            # intensity_image=img_raw,
                            properties=('label',
                                        'area',
                                        'centroid',
                                        'axis_major_length',
                                        'axis_minor_length',
                                        'eccentricity',
                                        'orientation',
                                        'slice',
                                        # 'extent',
                                        'equivalent_diameter_area',))
                                        #   'intensity_max',
                                        #   'intensity_min',
                                        #   'intensity_mean',))
                                        #   'image',))
    measurements_outer = pd.DataFrame(table)

    if plot_all == True:
        # now do the same for every outer cell:
        # plot centroids, measurements, major and minor axis on image to sanity check
        regions = regionprops(outer_labeled)

        fig, ax = plt.subplots(figsize=(8,8))
        ax.imshow(outer_labeled.astype('bool'), cmap='gray')

        for props in regions:
            y0, x0 = props.centroid
            orientation = props.orientation
            x1 = x0 + m.cos(orientation) * 0.5 * props.axis_minor_length
            y1 = y0 - m.sin(orientation) * 0.5 * props.axis_minor_length
            x2 = x0 - m.sin(orientation) * 0.5 * props.axis_major_length
            y2 = y0 - m.cos(orientation) * 0.5 * props.axis_major_length

            ax.plot((x0, x1), (y0, y1), '-r', linewidth=2.5)
            ax.plot((x0, x2), (y0, y2), '-r', linewidth=2.5)
            ax.plot(x0, y0, '.g', markersize=10)

            minr, minc, maxr, maxc = props.bbox
            bx = (minc, maxc, maxc, minc, minc)
            by = (minr, minr, maxr, maxr, minr)
            ax.plot(bx, by, '-b', linewidth=1.5)

        ax.set_axis_off()
        ax.axis((0, 1000, 1000, 0))
        plt.show()

    #re-label columns so they are accurate
    measurements_outer.columns = ['outer_' + col for col in measurements_outer.columns]
    measurements_inner.columns = ['inner_' + col for col in measurements_inner.columns]

    #put them in a single dataframe
    measurements = pd.concat([measurements_inner,measurements_outer],keys=['inner', 'outer'],axis=1)

    #add gratio to combined dataframe
    measurements['gratio'] = measurements['inner']['inner_axis_minor_length']/measurements['outer']['outer_axis_minor_length']
    measurements = measurements[measurements['gratio'] <= 1]

    #add orientation and eccentricity means between fiber and myelin. Also convert orientation to degrees from rad.
    measurements['orientation_mean_deg'] = np.degrees((measurements['inner']['inner_orientation']+measurements['outer']['outer_orientation'])/2)
    measurements['eccentricity_mean'] = ((measurements['inner']['inner_eccentricity']+measurements['outer']['outer_eccentricity'])/2)

    measurements.replace([np.inf, -np.inf], np.nan, inplace=True)
    if i == 0:
        measurements_all = measurements.copy()
    
    measurements_all = pd.concat([measurements_all,measurements],ignore_index=True)
    
    # Plot the histogram of g ratios
    plt.hist(measurements['gratio'],bins=50, edgecolor='black', color='darkblue', alpha=0.6)

    # Customize plot elements
    plt.ylabel('Number of cells')
    plt.xlabel('G Ratio')
    plt.grid(True, linestyle='--', alpha=0.8, which='both')
    # plt.xticks(np.arange(0, 1, 5))
    # plt.yticks(np.arange(0, 21, 2))

    # Add additional plot elements
    plt.axvline(measurements['gratio'].mean(), color='red', linestyle='--', label='Mean')
    plt.axvline(measurements['gratio'].median(), color='darkred', linestyle='--', label='Median')
    plt.legend()

    # Show the plot
    plt.tight_layout()
    plt.savefig(path_results + file + "_g_ratio.png",dpi=500)
    plt.show()

    if troubleshoot_small_big_gratios == True:
        ### troubleshoot 0/1 g ratio artifact
        measurements_smallgratio_slices = measurements[measurements['gratio'] < 0.3]
        measurements_biggratio_slices = measurements[measurements['gratio'] > 0.9]
        slice_list_big = measurements_biggratio_slices['inner']['inner_slice'].tolist()
        slice_list_small = measurements_smallgratio_slices['inner']['inner_slice'].tolist()
        p = 5 # p for padding
        for yx in slice_list_small:
            print('plotting small g ratios')
            print(str(yx))
            yx = find_numbers(str(yx)) # converts slice object to a list of numbers
            outer_eroded = ndi.binary_erosion(outer_labeled[yx[0]-p:yx[1]+p,yx[2]-p:yx[3]+p],structure=disk(1))
            ero = np.logical_xor(outer_labeled[yx[0]-p:yx[1]+p,yx[2]-p:yx[3]+p].astype('bool'),outer_eroded)
            plt.imshow(np.ma.array(inner_labeled[yx[0]-p:yx[1]+p,yx[2]-p:yx[3]+p] + outer_labeled[yx[0]-p:yx[1]+p,yx[2]-p:yx[3]+p],
                                   mask=(inner_labeled[yx[0]-p:yx[1]+p,yx[2]-p:yx[3]+p] + outer_labeled[yx[0]-p:yx[1]+p,yx[2]-p:yx[3]+p]) == 0),
                                   cmap=glasbey,
                                   vmin=vmin,vmax=vmax)
            plt.imshow(np.ma.array(ero, mask=(ero) == 0), interpolation='None', cmap='gray',)
            plt.axis('off')
            plt.savefig(path_results + "\\_gratio_small\\" + str(yx) + file + ".png")
            plt.show()

        for yx in slice_list_big:
            print('plotting large g ratios')
            print(str(yx))
            yx = find_numbers(str(yx)) # converts slice object to a list of numbers
            outer_eroded = ndi.binary_erosion(outer_labeled[yx[0]-p:yx[1]+p,yx[2]-p:yx[3]+p],structure=disk(1))
            ero = np.logical_xor(outer_labeled[yx[0]-p:yx[1]+p,yx[2]-p:yx[3]+p].astype('bool'),outer_eroded)
            plt.imshow(np.ma.array(inner_labeled[yx[0]-p:yx[1]+p,yx[2]-p:yx[3]+p] + outer_labeled[yx[0]-p:yx[1]+p,yx[2]-p:yx[3]+p],
                                   mask=(inner_labeled[yx[0]-p:yx[1]+p,yx[2]-p:yx[3]+p] + outer_labeled[yx[0]-p:yx[1]+p,yx[2]-p:yx[3]+p]) == 0),
                                   cmap=glasbey,
                                   vmin=vmin,vmax=vmax)
            plt.imshow(np.ma.array(ero, mask=(ero) == 0), interpolation='None', cmap='gray',)
            plt.axis('off')
            plt.savefig(path_results + "\\_gratio_big\\" + str(yx) + file + ".png")
            plt.show()
    i +=1

In [ ]:
### TODO:
# Data cleanup.

In [ ]:
# Plot the histogram of g ratios
plt.hist(measurements_all['gratio'],bins=50, edgecolor='black', color='darkblue', alpha=0.6)

# Customize plot elements
plt.ylabel('Number of cells')
plt.xlabel('G Ratio')
plt.grid(True, linestyle='--', alpha=0.8, which='both')
# plt.xticks(np.arange(0, 1, 5))
# plt.yticks(np.arange(0, 21, 2))

# Add additional plot elements
plt.axvline(measurements_all['gratio'].mean(), color='red', linestyle='--', label='Mean')
plt.axvline(measurements_all['gratio'].median(), color='darkred', linestyle='--', label='Median')
plt.legend()

# Show the plot
plt.tight_layout()
plt.savefig(path_results + file + "_all_g_ratio.png",dpi=500)
plt.show()

In [ ]:
# Plot the histogram of eccentricity
plt.hist(measurements_all['inner']['inner_eccentricity'],bins=50, edgecolor='black', color='darkblue', alpha=0.6)

# Customize plot elements
plt.ylabel('Number of cells')
plt.xlabel('Inner eccentricity')
plt.grid(True, linestyle='--', alpha=0.8, which='both')
# plt.xticks(np.arange(0, 1, 5))
# plt.yticks(np.arange(0, 21, 2))

# Add additional plot elements
plt.axvline(measurements_all['inner']['inner_eccentricity'].mean(), color='red', linestyle='--', label='Mean')
plt.axvline(measurements_all['inner']['inner_eccentricity'].median(), color='darkred', linestyle='--', label='Median')
plt.legend()

# Show the plot
plt.tight_layout()
plt.savefig(path_results + file + "_all_eccentricity.png",dpi=500)
plt.show()

In [ ]:
# Plot the histogram of g ratios
plt.hist(measurements_all['outer']['outer_eccentricity'],bins=50, edgecolor='black', color='darkblue', alpha=0.6)

# Customize plot elements
plt.ylabel('Number of cells')
plt.xlabel('Outer Eccentricity')
plt.grid(True, linestyle='--', alpha=0.8, which='both')
# plt.xticks(np.arange(0, 1, 5))
# plt.yticks(np.arange(0, 21, 2))

# Add additional plot elements
plt.axvline(measurements_all['outer']['outer_eccentricity'].mean(), color='red', linestyle='--', label='Mean')
plt.axvline(measurements_all['outer']['outer_eccentricity'].median(), color='darkred', linestyle='--', label='Median')
plt.legend()

# Show the plot
plt.tight_layout()
plt.savefig(path_results + file + "_all_eccentricity.png",dpi=500)
plt.show()

In [ ]:
# plot a polar scatterplot. In this scatterplot, direction (theta) is determined by orientation, 
# displacement (r) is determined by eccentricity
# color is determined by the g ratio, but can be switched to e.g. outer area (commented out line)

fig = px.scatter_polar(measurements_all, 
                    r="eccentricity_mean", 
                    theta="orientation_mean_deg", 
                    # color=measurements_all["gratio"],
                    color=measurements_all["inner"]["inner_area"],
                    template="plotly_dark",
                    range_theta=(0,180),
                    range_r=(0,1),
                    title="Orientation (radial axis), Eccentricity (angular displacement), fiber area (color)")
pio.write_image(fig, path_results + file + "_Orientation_vs_area.png")
fig.show()

In [ ]:
# plot a polar scatterplot. In this scatterplot, direction (theta) is determined by orientation, 
# displacement (r) is determined by eccentricity
# color is determined by the g ratio, but can be switched to e.g. outer area (commented out line)

# Create a boolean mask based on the condition
mask = measurements_all['inner']['inner_area'] > 5000 & (measurements_all['inner']['inner_area'] < 15000)
# Apply the mask to the DataFrame to filter the entries
measurements_all_filtered = measurements_all[mask]

fig = px.scatter_polar(measurements_all_filtered, 
                    r="eccentricity_mean", 
                    theta="orientation_mean_deg", 
                    # color=measurements_all["gratio"],
                    color=measurements_all_filtered["inner"]["inner_area"],
                    template="plotly_dark",
                    range_theta=(0,180),
                    range_r=(0,1),
                    title="Orientation (radial axis), Eccentricity (angular displacement), fiber area (color)")
pio.write_image(fig, path_results + file + "_Orientation_vs_area_filtered.png")
fig.show()

In [ ]:
# plot a polar scatterplot. In this scatterplot, direction (theta) is determined by orientation, 
# displacement (r) is determined by eccentricity
# color is determined by the g ratio, but can be switched to e.g. outer area (commented out line)
fig = px.scatter_polar(measurements_all, 
                    r="eccentricity_mean", 
                    theta="orientation_mean_deg", 
                    color=measurements_all["gratio"],
                    # color=measurements_all["inner"]["inner_area"],
                    template="plotly_dark",
                    range_theta=(0,180),
                    range_r=(0,1),
                    title="Orientation (radial axis), Eccentricity (angular displacement), gratio (color)")
pio.write_image(fig, path_results + file + "_Orientation_vs_gratio.png")
fig.show()

#### Save results to disk

In [ ]:
tf.imwrite(path_results + file + "_inner.tif", outer_labeled)
tf.imwrite(path_results + file + "_outer.tif", inner_labeled)
tf.imwrite(path_results + file + "_raw_prediction.tif", img)
# tf.imwrite(path_results + "raw.tif", img_raw)
measurements_all.to_csv(path_results + "all_results.csv")
measurements_all.to_excel(path_results + "all_results.xlsx")


In [ ]:
# import stackview
# stackview.curtain(img,(inner_labeled + outer_labeled),continuous_update=True)

#### code graveyard
resurrect if needed

In [ ]:
# ### find corresponding outer label to each inner cell

# # # make copy of inner labels, but dilated
# # mask_dil = ndi.binary_dilation(mask_labeled,iterations=1,structure=(disk(1)))

# # print('number of cells before dilation: ' + str(ndi.label(mask_labeled)[1]))
# # print('number of cells after dilation: ' + str(ndi.label(mask_dil)[1]))

# # # watershed to segregate erroneously joined cells
# # mask_dil_relabeled = resolve_undersegmentation(mask_dil,mask_labeled)
# # print('number of cells after resolving undersegmentation: ' + str(np.max(mask_labeled)))

# # initialize copy of outer channel and counters
# outer_labeled = np.copy(outer)
# outer_labeled[outer_labeled==True] = 65000
# no_myelin_counter = 0
# myelin_counter = 0

# # find best fitting outer cell for each inner cell
# print ('Finding corresponding myelin structures.. ')
# for idx in tqdm.tqdm(np.unique(mask_labeled)):
#     # find overlap of given cell and all outer structures
#     if idx == 0:
#         continue

#     # create mask for currently active cell
#     current_id_mask = mask_labeled==idx 

#     # dilate this mask so it overlaps with neighbouring cells
#     current_id_mask = ndi.binary_dilation(current_id_mask,iterations=1,structure=(disk(5)))

#     # check if it overlaps with the outer channel
#     overlap = np.logical_and(outer,current_id_mask)
#     # plt.imshow(overlap)
#     # plt.show()
#     if overlap.max() == True:
#         #find middle point of overlap
#         overlap_dist = ndi.distance_transform_bf(overlap)
#         max_list = np.argwhere(overlap_dist==np.max(overlap_dist))
#         if max_list.shape[0]>1:
#             # choose random point within the overlap to flood fill the outer channel
#             yx_max = max_list[r.randint(0,max_list.shape[0]-1)]
#         else:
#             yx_max = max_list[0]
        
#         # fill the according spot in outer_labeled with the right idx value
#         outer_labeled = skimage.segmentation.flood_fill(outer_labeled,
#                                                        (yx_max[0],yx_max[1]),
#                                                         new_value=idx,)
#         myelin_counter += 1 
    
#     if overlap.max() == False:
#         no_myelin_counter += 1
    
#     # if there are multiple instances of it being connected, find largest overlap and use that one

# print ('No myelin found for cells: ' + str(no_myelin_counter))
# print ('Myelin found for cells: ' + str(myelin_counter))

In [ ]:
# circle_mask = np.zeros_like(max_labeled)

# for id in range(max_labeled.max()+1):
#     if id == 0: continue
#     # get x and y coordinates of where the cell is
#     yx = np.array(np.nonzero(max_labeled==id))
#     if yx.shape != (2,1): continue #### for some reason, some coordinates are of shape (2,2 instead of 2,1??)
#     # get corresponding circle area
#     area = measurements["area"][id]
#     # draw circle with radius that corresponds to the area enclosed by the cell
#     r = round(m.sqrt(area/m.pi))
#     # draw circle boundaries
#     c1 = skimage.morphology.disk(radius=r+3)*id # outer circle
#     c2 = skimage.morphology.disk(radius=r)*id # inner circle

#     circle_mask[int(yx[0]-c1.shape[0]//2):int(yx[0]+c1.shape[0]//2+1),
#                 int(yx[1]-c1.shape[0]//2):int(yx[1]+c1.shape[0]//2+1)] += c1 # add outer circle

#     circle_mask[int(yx[0]-c2.shape[0]//2):int(yx[0]+c2.shape[0]//2+1),
#                 int(yx[1]-c2.shape[0]//2):int(yx[1]+c2.shape[0]//2+1)] -= c2 # subtract inner circle

# plt.figure(figsize=(8,8))
# plt.axis("off")
# plt.imshow(mask_labeled.astype("bool"),interpolation='none',cmap="gray")
# plt.imshow(np.ma.array(circle_mask, mask=circle_mask==0), interpolation='none', cmap='tab20')
# plt.colorbar(fraction=0.046, pad=0.04)
# plt.show()

In [ ]:
# def resolve_undersegmentation(outer_labels,inner_labels):
#     '''
#     resolves undersegmented cells with 'kissing' outer boundaries by watershed.
    
#     outer_labels = outer labels with undersegmented kissing cells that need to be segmented
#     inner_labels = inner labels without undersegmented kissing cells

#     returns two arrays of the same shape, with uniform labels across both images. 
#     '''
#     from skimage.segmentation import watershed
#     from skimage.feature import peak_local_max
#     outer_labels = outer_labels.astype('bool')
#     inner_labels = inner_labels.astype('bool')
#     # do distance transformation of combined binary image of outer+inner
#     seg_dist = ndi.distance_transform_edt(outer_labels+inner_labels)
#     # Generate the markers as local maxima of the distance to the background, restricted to 1 marker per label
#     coords = peak_local_max(seg_dist, footprint=np.ones((3, 3)), labels=ndi.label(inner_labels)[0], num_peaks_per_label=1)
#     mask = np.zeros(seg_dist.shape, dtype=bool)
#     mask[tuple(coords.T)] = True
#     markers, n = ndi.label(mask)
#     outer_cells = watershed(-seg_dist, markers, mask=(outer_labels+inner_labels))
#     inner_cells = np.copy(outer_cells)
#     inner_cells[inner_labels==0] = 0
#     outer_cells[inner_labels==True] = 0
#     return outer_cells,inner_cells